In [ ]:
import tempfile
import time
import torch
import numpy as np
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass
from transformers import AutoTokenizer, AutoModelForCausalLM
from evaluate import load
import shutil
import os
import subprocess
import json

# TRANSPILADOR REGEX

In [ ]:
!git clone https://github.com/Berly01/Transpilador-Compiladores.git

Cloning into 'Transpilador-Compiladores'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 33 (delta 12), reused 27 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (33/33), 238.06 KiB | 1.25 MiB/s, done.
Resolving deltas: 100% (12/12), done.


In [ ]:
%cd Transpilador-Compiladores
%mkdir build

/content/Transpilador-Compiladores


In [ ]:
!cmake -S . -B build
!cmake --build build

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Configuring done (0.7s)
-- Generating done (0.0s)
-- Build files have been written to: /content/Transpilador-Compiladores/build
[ 50%] Building CXX object Transpiler/CMakeFiles/Transpiler.dir/Transpiler.cpp.o
[100%] Linking CXX executable Transpiler
[100%] Built target Transpiler


# TRANSPILADOR LLM

In [ ]:
%cd /content

/content


In [ ]:
model_name = "Berly0/CodeLlama-7b-hf-c-to-cpp"

In [ ]:
def load_model():
  print(f"Cargando tokenizer para {model_name}...")
  tokenizer = AutoTokenizer.from_pretrained(model_name)

  print(f"Cargando modelo para {model_name}...")

  model = AutoModelForCausalLM.from_pretrained(
      model_name,
      torch_dtype=torch.float16,
      device_map="auto"
  )

  print("Modelo y tokenizer cargados exitosamente.")
  return model, tokenizer

In [ ]:
def load_test_dataset(test_dataset_path) -> List[Dict]:
  """Carga el dataset de prueba"""
  data = []
  with open(test_dataset_path, 'r', encoding='utf-8') as f:
    for line in f:
      data.append(json.loads(line.strip()))
  return data

In [ ]:
def create_prompt(source_code, mode="llm"):
    """Crea el prompt para el entrenamiento"""
    if mode == "llm":
        prompt = f"""Translate the following code from C to C++:
code C:
{source_code}

code C++:"""
    elif mode == "regex+llm":
        prompt = f"""The following source code is mostly written in C, but contains some partial constructs or C++ style. Translate it completely to C++.:

code C:
{source_code}

code C++:"""

    return prompt

In [ ]:
model, tokenizer = load_model()

Cargando tokenizer para Berly0/CodeLlama-7b-hf-c-to-cpp...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/160M [00:00<?, ?B/s]

Modelo y tokenizer cargados exitosamente.


In [ ]:
def test_model(model, c_code, mode='llm', temperature=0.7, max_new_tokens=256):
    """Funcion para probar el modelo entrenado"""

    prompt = create_prompt(c_code, mode)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    #cpp_start = generated_text.find("C++ code:") + len("C++ code:")
    #cpp_code = generated_text[cpp_start:].strip()

    return generated_text

# REGEX + LLM

In [ ]:
def read_text_file(file_path):
  try:
    with open(file_path, 'r', encoding='utf-8') as f:
      content = f.read()
    return content
  except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found.")
    return None
  except Exception as e:
    print(f"An error occurred while reading the file: {e}")
    return None

In [ ]:
code1 = r"""#include <stdio.h>
#include <stdlib.h>

// Define some constants using #define
#define INITIAL_SIZE 0
#define VALUE_COUNT 5
#define VALUE_1 10
#define VALUE_2 20
#define VALUE_3 30
#define VALUE_4 40
#define VALUE_5 50

// Function to add an integer to the list
int* add_to_list(int* list, int* size, int value) {
    (*size)++;
    int* temp = realloc(list, (*size) * sizeof(int));
    if (temp == NULL) {
        printf("Memory allocation failed!\n");
        free(list);
        exit(1);
    }
    list = temp;
    list[(*size) - 1] = value;
    return list;
}

// Function to print the list
void print_list(int* list, int size) {
    printf("Current List (%d items): ", size);
    for (int i = 0; i < size; i++) {
        printf("%d ", list[i]);
    }
    printf("\n");
}

int main() {
    int* list = NULL;
    int size = INITIAL_SIZE;

    // Use defined constants for values to add
    int values_to_add[VALUE_COUNT] = {
        VALUE_1, VALUE_2, VALUE_3, VALUE_4, VALUE_5
    };

    for (int i = 0; i < VALUE_COUNT; i++) {
        list = add_to_list(list, &size, values_to_add[i]);
        print_list(list, size);
    }

    printf("\nFinal list:\n");
    print_list(list, size);

    free(list);
    printf("Memory successfully freed.\n");

    return 0;
}
"""

In [ ]:
def compile_cpp_program(cpp_file_path):
    """
    Comprueba si un programa de C++ se compila correctamente.
    """
    try:
        base_name = os.path.basename(cpp_file_path)
        file_name_without_extension, _ = os.path.splitext(base_name)

        subprocess.run(["g++", cpp_file_path, "-o", file_name_without_extension], check=True, capture_output=True, text=True)
        return True
    except subprocess.CalledProcessError as e:
        return False
    except FileNotFoundError:
        print("Error: g++ compiler not found. Make sure it is installed and in your PATH.")
        return False


def execute_cpp_program(output_file_path):
    """
    Comprueba si un programa C++ compilado se ejecuta correctamente.
    """
    try:
        subprocess.run([output_file_path], check=True, capture_output=True, text=True, timeout=10)
        return True
    except subprocess.CalledProcessError as e:
        return False
    except FileNotFoundError:
        print(f"Error: The executable '{output_file_path}' was not found.")
        return False
    except subprocess.TimeoutExpired:
        return False
    except Exception as e:
        return False

In [ ]:
def extract_cpp_code(text):
  start_marker = "code C++:"
  end_marker = "return 0;"

  start_index = text.find(start_marker)
  if start_index == -1:
    print(f"Start marker '{start_marker}' not found.")
    return None
  start_index += len(start_marker)

  end_index = text.find(end_marker, start_index)
  if end_index == -1:
    print(f"End marker '{end_marker}' not found after start marker.")
    return None

  extracted_text = text[start_index:end_index].strip()

  return extracted_text + '\n\n\treturn 0;\n}'

def translate(input: str, output: str, mode='regex+llm'):
  command = [
    "/content/Transpilador-Compiladores/build/Transpiler/Transpiler",
    input,
    output
  ]

  try:
      subprocess.run(command, check=True)
      first_translation = read_text_file(output)
      result = test_model(
          model=model,
          c_code=first_translation,
          mode=mode,
          temperature=0.7,
          max_new_tokens=1024)

      second_translation = extract_cpp_code(result)

      base_name = os.path.basename(input)
      file_name_without_extension, _ = os.path.splitext(base_name)

      path = f'/content/{file_name_without_extension}_translation.cpp'
      with open(path, 'w') as f:
        f.write(second_translation)

  except subprocess.CalledProcessError as e:
      print(f"Error executing command: {e}")
  except FileNotFoundError:
      print("Error: The command or one of the input files was not found.")

In [ ]:
CODE1_PATH = '/content/code1.c'

with open(CODE1_PATH, 'w') as f:
    f.write(code1)

In [ ]:
translate(
    input='/content/code1.c',
    output='/content/code1-output.cpp',
    mode='regex+llm')

In [ ]:
if compile_cpp_program('/content/code1_translation.cpp'):
  print('Compilación exitosa')

Compilación exitosa


In [ ]:
if execute_cpp_program('/content/code1_translation'):
  print('Ejecución exitosa')

Ejecución exitosa


In [ ]:
print(read_text_file('/content/code1_translation.cpp'))

#include <iostream>
#include <string>
#include <array>
#include <cstdlib>
#include <memory>
#include <vector>

using namespace std;

constexpr int INITIAL_SIZE = 0;
constexpr int VALUE_COUNT = 5;
constexpr int VALUE_1 = 10;
constexpr int VALUE_2 = 20;
constexpr int VALUE_3 = 30;
constexpr int VALUE_4 = 40;
constexpr int VALUE_5 = 50;

// Function to add an integer to the list
vector<int> add_to_list(vector<int> list, int value) {
    list.push_back(value);
    return list;
}

// Function to print the list
void print_list(vector<int> list) {
    cout << "Current List (" << list.size() << " items): ";
    for (int i = 0; i < list.size(); i++) {
        cout << list[i] << " ";
    }
    cout << "\n";
}

int main() {
    vector<int> list = {};

    // Use defined constants for values to add
    int values_to_add[VALUE_COUNT] = {
        VALUE_1, VALUE_2, VALUE_3, VALUE_4, VALUE_5
    };

    for (int i = 0; i < VALUE_COUNT; i++) {
        list = add_to_list(list, values_to_add[i]);
       